# Transferlearning

We have seen that convolutional neural networks are able to extract abstract spatial features that are common within the training set and are sufficient to classify images into different classes. Looking at the kernels that resulted from the MNIST dataset it is hard to imagine that these kernels could be of general use, however. This is not the case for more general images such as the CIFAR dataset. One can imagine, that a detector that has been trained on a very large number (60000) of very general images actually does lead to a fairly general set of kernels. If that is so, a network could be trained on additional or new images without actually touching the convolutional part. This is known as <i>Transferlearning</i>. 

In [1]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

import matplotlib.pyplot as plt

for I in range(4):
    for J in range(4):
        plt.subplot(4,4,I*4+J+1)
        plt.imshow(x_train[I*4+J,:,:,:])

Using TensorFlow backend.


Training a classifer with 79% accuracy for the CIFAR dataset took a little less than an hour on a simple computer. What about a set of kernels that have been trained on millions of images that takes weeks or months to train? How about a set of kernels that has been trained to extract patterns millions of hours of human speech or even video? Making such kernels available might enable individuals and small companies to leverage the power of deep learning for their applications without the resources that are needed to train networks using so much data or even collect them. 

Although the distinction between the "convolutional" and the "dense" part is intuitive, <b>what</b> part of the network to choose is much less obvious in practice. It is also rarely the case that the application to which we want to transfer to is using the exact same data as input. For example, the CIFAR classifier has been trained on 32x32, square images. How can this be adapted to a dataset with larger images (except for throwing away the additional data)? What about images that have been recorded by a different camera and specific lightning conditions? Does a model that has been trained on outdoor image contain anything that can be reused for indoor images? In the worst case, transferring parts of a network make things worse, also known as <i>negative transfer</i>, making it harder for training to recover than training from scratch. 

The field of transfer learning is large, the questions treated therein are subtle, and not limited to deep learning. A good overview is provided in the article <a href="https://towardsdatascience.com/a-comprehensive-hands-on-guide-to-transfer-learning-with-real-world-applications-in-deep-learning-212bf3b2f27a">A comprehensive hands on guide to transfer learning with real world applications in deep learning</a> and the research paper <a href="https://www.cse.ust.hk/~qyang/Docs/2009/tkde_transfer_learning.pdf">A survey on transfer learning</a>.

Transfer learning in a deep learning context is known as <i>inductive learning</i>, which in contrast to <i>unsupervised transfer learning</i> does have labeled data in a target domain. Depending on whether labeled data is available in a source domain or not, we are talking about <i>multi-task learning</i> or <i>self-taught learning</i>. The latter, self-taught learning, is a situation in which we obtain a pre-trained network and apply it to a similar category, but do not take advantage of the original training set. 

## Pre-trained Models as Feature Extractors

One of the most prominent pre-trained models are VGG (University of Oxford), ImageNet (Google) and Inception (Microsoft).